## Neural Network implementation
In this notebook, we will implement a neural network with one hidden layer from scratch. It will be used for a regression task. To test it, we will try to train the network to predict the function $y=\sin(2 \pi x)$.

First we load the required libraries. We'll use `numpy` and `matplotlib`. The neural net is fully implemented in `numpy`, while `matplotlib` is used for visualisation. We also import `AERO40041` which provides some testing functions that provide feedback on your implementation. After runnig certain cells, you will see "Passed!" if your code is working as expected, or a message to check it if it doesn't seem to be working.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import AERO40041

### Parameter matricies setup

The parameters of our network are $\mathbf{W}^{[1]}$, $\mathbf{W}^{[2]}$, $\mathbf{b}^{[1]}$ and $\mathbf{b}^{[2]}$. We define a function that initialises these to the correct sizes based on the number of neurons in each layer. The weights are initialised to some small random number. This random initialisation breaks the symmetry. If all weights were initialized to the same value (e.g., zero), the neurons in a layer would be computing the same thing, leading to a redundancy in the network that prevents effective learning.

Each weight is initialised to a small random number. There are more sophisticated methods of choosing initial weights than this, but for the sake of simplicity, this will do fine. 

The size and shapes of $\mathbf{W}$ and $\mathbf{b}$ are as follows:

$$\mathbf{W}^{[1]} \equiv 
\begin{bmatrix}
w_{1,1}^{[1]} & w_{1,2}^{[1]} & \cdots & w_{1,R^{[0]}}^{[1]}\\
w_{2,1}^{[1]} & w_{2,2}^{[1]} & \cdots & w_{2,R^{[0]}}^{[1]}\\
\vdots & \vdots & \ddots & \vdots \\
w_{R^{[1]},1}^{[1]} & w_{R^{[1]},2}^{[1]} & \cdots & w_{R^{[1]},R^{[0]}}^{[1]}
\end{bmatrix} 
\hspace{1cm} \mathbf{W}^{[2]} \equiv 
\begin{bmatrix}
w_{1,1}^{[2]} & w_{1,2}^{[2]} & \cdots & w_{1,R^{[1]}}^{[2]}\\
w_{2,1}^{[2]} & w_{2,2}^{[2]} & \cdots & w_{2,R^{[1]}}^{[2]}\\
\vdots & \vdots & \ddots & \vdots \\
w_{R^{[2]},1}^{[2]} & w_{R^{[2]},2}^{[2]} & \cdots & w_{R^{[2]},R^{[1]}}^{[2]}
\end{bmatrix}$$

$$\mathbf{b}^{(1)} \equiv 
\begin{bmatrix}
b_{1}^{[1]} \\
b_{2}^{[1]} \\
\vdots  \\
b_{R^{[1]}}
\end{bmatrix} 
\hspace{1cm} \mathbf{b}^{(2)} \equiv 
\begin{bmatrix}
b_{1}^{[2]} \\
b_{2}^{[2]} \\
\vdots  \\
b_{R^{[2]}}
\end{bmatrix}$$

where $R^{[i]}$ is the size of the $i^{th}$ layer ($i=0$ is the input, $i=1$ is the hidden layer, and $i=2$ is the output).

In [2]:
def initialise_params(N_Neurons) :
    #fix the pseudo-random number generator seed so rerunning the code produces the same output.
    rng = np.random.default_rng(seed=123456) 

    #initialise the weights to a small random number between -0.015 and 0.015.
    W1 = 0.03*rng.random((N_Neurons[1], N_Neurons[0])) -0.015
    W2 = 0.03*rng.random((N_Neurons[2], N_Neurons[1])) -0.015
    b1 = np.zeros((N_Neurons[1],1))
    b2 = np.zeros((N_Neurons[2],1))

    return W1, W2, b1, b2

Let's test our above code for a network with three features (so the input size is three), one output, and 20 neurons in the hidden layer:

<img src="https://raw.githubusercontent.com/AlexSkillen/AlexSkillen.github.io/refs/heads/main/AERO40041/images/NN1.png" width=600>

In [3]:
W1, W2, b1, b2 = initialise_params([3,20,1])
print("The shape of W1 is: ", W1.shape)
print("The shape of W2 is: ", W2.shape)
print("The shape of b1 is: ", b1.shape)
print("The shape of b2 is: ", b2.shape)

The shape of W1 is:  (20, 3)
The shape of W2 is:  (1, 20)
The shape of b1 is:  (20, 1)
The shape of b2 is:  (1, 1)


Check these are the sizes expected. Note a shape of (1, 20) means a row vector of length 20. A shape of (20, 1) is a column vector of length 20. See below:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Matrix.svg/1920px-Matrix.svg.png" width=400>

### Activation functions
We will use a $\tanh$ activation function in the hidden layer.

The gradient $\frac{d}{dx}\tanh(x) = 1-\tanh^2(x)$. This will be needed for backpropagation so we implement it in the cell below.

In [4]:
def Grad_tanh(x) :
    return 1. - np.tanh(x)**2

As this is a regression task, we will use linear activation for the output layer. Linear activation means the output is equal to the input, i.e., $f(n) = n$. It's gradient is therefore just 1.  

### Forward propagation

Forward propagation can be expressed as follows:

$$\mathbf{a}^{[i]} = f^{[i]}\left(\mathbf{W}^{[i]}\mathbf{p}^{[i]} + \mathbf{b}^{[i]}\right)$$

$$\mathbf{p}^{[i]} = 
\begin{cases}
\mathbf{x}_k &\text{for the first hidden layer}\\
\mathbf{a}^{[i-1]} &\text{otherwise}
\end{cases}
$$

where $\mathbf{a}^{[i]}$ is the vector of activated outputs of layer $i$, $\mathbf{W}^{[i]}$ is the weights matrix for layer $i$, $\mathbf{b}^{[i]}$ is the bias vector, and $\mathbf{p}^{[i]}$ is the input to the layer (i.e. the feature vector $\mathbf{x}_k$ if we are considering the first layer, where $k$ is the example number, or the output of the previous layer for all subsequent layers).

In the function below, implement the forward propagation algorithm. You will find the function `np.matmul` useful for the matrix multiplication of two matrices (or vectors), and `np.tanh` for the tanh function. Your forward function should return the vector of activation potentials and vector of activated outputs of each layer as these will be needed for backpropagation later.

In [7]:
def forward( x, W1, W2, b1, b2 ) :
    """
    Input: 
        x (a feature vector from a single example)
        W1 (the weights matrix for the hidden layer)
        W2 (the weights matrix for the output layer)
        b1 (the bias vector for the hidden layer)
        b2 (the bias vector for the output layer)

    Outputs:
        n1 (the activation potential for the hidden layer)
        a1 (the activated output of the hidden layer)
        n2 (the activation potential for the output layer)
        a2 (the activated output of the output layer)
    """ 
    
    n1 = np.matmul(W1, x) + b1
    a1 = np.tanh(n1)
    n2 = np.matmul(W2, a1) + b2
    a2 = n2
    return n1, a1, n2, a2

AERO40041.test_forward(forward)

Passed!


### Cost function

We will use the MSE loss. The cost function takes the full training set and loops over all the data to compute the cost. While this function is not strictly necessary (we only need its derivative for backpropagation), it is useful to evaluate how training progresses. The function takes in X and Y arguments. X is the full training set of features where X[0] is the first feature vector, X[1] is the second, etc.

Each element X[k] is the $k^{\mathrm{th}}$ feature vector in column format. I.e. 

$$X[k] = \begin{bmatrix}
x_1\\
x_2\\
\vdots\\
x_D 
\end{bmatrix}_k
$$

where $D$ is the number of features (also referred to as the dimension). 

The same applies for Y; Y[k] is also a vector in column format to account for the general case where the network outputs more than one value. The vector of labels Y[k] corresponds to the feature vector X[k], so X[1] is the feature vector associated with Y[1], X[100] is the feature vector associated with Y[100], etc.

To keep the code general and ensure it works no matter if we have a single feature, or a single output while still using `matmul`, we store each X[k] as a (1,1) matrix for the single feature case. Similarly for Y, we store each label as a (1,1) matrix for cases of scalar output.

Implement the MSE cost function below:


In [8]:
def cost(X, Y, W1, W2, b1, b2):
    """
    Input:
        X (the full training set of feature vectors)
        Y (the full training set of labels)
        W1 (the weights matrix for the hidden layer)
        W2 (the weights matrix for the output layer)
        b1 (the bias vector for the hidden layer)
        b2 (the bias vector for the output layer)

    Output:
        c (the cost)
    """
    
    c=0.
    
    for i in range(len(X)):
        _, _, _, a2 = forward(X[i], W1, W2, b1, b2)
        c = c + (Y[i] - a2)**2

    c = c/float(len(X))

    """
    In the case of more than one output, the above will compute a vector of MSE costs for each output. 
    In this example (and your coursework), since we have only one output, we could just return c calculated above
    directly, but to deal with the more general case, we will sum the MSE associated with each output to get
    the overall cost. This last step is optional if we always have single outputs.
    """

    c = np.sum(c)
    
    return c

AERO40041.test_cost(cost)

Passed!


### Backpropagation

The backpropagation equations are given below. Here we focus on the practical implementation. The theory will be covered in the lectures.

\begin{align}
\frac{\partial \mathcal{L}}{\partial \mathbf W^{[i]}} &=
%
\left[\frac{\partial \mathcal{L}}{\partial \mathbf{a}^{[i]}} \odot f^\prime(\mathbf{n}^{[i]}) \right] \mathbf{a}^{[i-1]T}
&
\text{Eq. 1}
\end{align}

\begin{align}
\frac{\partial \mathcal{L}}{\partial \mathbf b^{[i]}} &=
%
\frac{\partial \mathcal{L}}{\partial \mathbf{a}^{[i]}} \odot f^\prime(\mathbf{n}^{[i]})
& \text{Eq. 2}
\end{align}


where $\odot$ is the Hadamard product (i.e. multiply element-wise two vectors of the same size to produce a third vector of the same size). For example 

$$
\begin{bmatrix}
1 \\
2 \\
3 \\
\end{bmatrix}
\odot
\begin{bmatrix}
4 \\
5 \\
6 \\
\end{bmatrix}
=
\begin{bmatrix}
4 \\
10 \\
18 \\
\end{bmatrix}
$$

In python, the Hadamard product can be implemented with the `*` operator. i.e. `a*b` will return $\mathbf{a} \odot \mathbf{b}$ if `a` and `b` are both vectors. They can be column vectors, or row vectors (but not one of each).  

The partial derivative vector $\partial \mathcal{L} / \partial \mathbf{a}^{[L]}$ for the final layer is found by differentiation of the loss function. For all previous layers, it is found as: 


\begin{align}
\frac{\partial \mathcal{L}}{\partial \mathbf{a}^{[L]}}
&=
\mathbf{W}^{[L+1]T}
\left[\frac{\partial \mathcal{L}}{\partial \mathbf{a}^{[L+1]}} \odot f^\prime(\mathbf{n}^{[L+1]})\right]
& \text{Eq. 3}
\end{align}

Implement the backpropagation algorithm below. We will use Stochastic gradient descent, so gradients are computed with only one example. The function can take arguments `x`, `y`, `n1`, `a1`, `n2`, `a2`, `W1`, `W2`, `b1`, `b2` and `alpha` described in the template below.

Values for `n1`, `a1`, `n2`, `a2` will be found from the forward propagation you implemented earlier.

Your function should also update the parameters by SGD.

In [ ]:
def backward(x, y, n1, a1, n2, a2, W1, W2, b1, b2, alpha) :
    """
        ==========================================================
    Inputs:
        x (a single feature vector)
        y (a single label)
        n1 (the activation potential for the hidden layer)
        a1 (the activated output of the hidden layer)
        n2 (the activation potential for the output layer)
        a2 (the activated output of the output layer)
        W1 (the weights matrix for the hidden layer in the data structure format as specified by initialise_params)
        W2 (the weights matrix for the output layer in the data structure format as specified by initialise_params)
        b1 (the bias vector for the hidden layer in the data structure format as specified by initialise_params)
        b2 (the bias vector for the output layer in the data structure format as specified by initialise_params)
        alpha (the learning rate)
    Outputs:
        W1 (the weights matrix for the hidden layer after SGD update)
        W2 (the weights matrix for the output layer after SGD update)
        b1 (the bias vector for the hidden layer after SGD update)
        b2 (the bias vector for the output layer after SGD update)
        ==========================================================
    """

    """
    Step 1. Compute the partial derivative of the Loss w.r.t. the output
    Note we do not need to sum this up to get the cost function because we are using SGD without any mini-batching.
    """
    
    dL_da2 = -2 * (y - a2)
    

    """
    Step 2. Find the gradients of the activation functions evaluated at n. These appear in Eq. 1, 2 and 3 as f primed 
    where f prime is the gradient of the activation function. For the output layer, we use linear 
    activation so its derivative is 1. We use np.ones_like to get a vector of ones the same shape as dL_da2. 
    For the hidden layer, we call the Grad_tanh function we defined above:
    """
    
    f2_prime_n2 = np.ones_like(dL_da2)  
    f1_prime_n1 = Grad_tanh(n1)

    
    """
    Step 3. Get the partial derivatives dL/da1 for the hidden layer. This is found from Eq. 3 above.
    """

    dL_da1 = None #replace None with your implementation

    """
    Step 4.
    Now we have all the individual terms appearing in Eq 1, 2 and 3, we can compute dL/dW and dL/db for each layer
    from Eq 1 and Eq2.
    """


    #From Eq. 1:
    dL_dW2 = None #replace None with your implementation
    dL_dW1 = None #replace None with your implementation

    #from Eq. 2:
    dL_db2 = None #replace None with your implementation
    dL_db1 = None #replace None with your implementation

    """
    Step 5:
    Now we have the gradients, we apply stochastic gradient descent to update the parameters:
    """
        
    W1 = None #replace None with your implementation
    W2 = None #replace None with your implementation
    b1 = None #replace None with your implementation
    b2 = None #replace None with your implementation

    """
    Finally, we can return the updated parameters!
    """
    return W1, W2, b1, b2
    
AERO40041.test_backward(backward)

This has not passed our test. This does not necessarily mean it is incorrect (for instance if you have changed the order of arguments the test might produce a false negative). It is recommended you recheck you code or ask for assistance if you cannot spot any issues.


### Training loop
In the cell below, we implement a training loop that loops over all examples in the training set in random order (this is important for SGD) and calls `forward` and then `backward` successively. This whole operation is repeated `epoch` times, where `epoch` is an argument to the function that defaults to 10, but can be set to any value when the function is called. 

The line `np.arange(len(X))` outputs a vector $[0, 1, 2, 3 \cdots (N-1)]$. This vector is rearranged into a random order with the line `np.random.shuffle(indices)`. The data is then copied into X_shuff and Y_shuff in this random order. We then loop over the shuffled data calling our forward and backward functions on a single example (i.e. SGD, no mini-batching).


In [8]:
def train(X, Y, W1, W2, b1, b2, alpha = 0.01, epoch = 10):
    for j in range(epoch):
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X_shuff = X[indices]
        Y_shuff = Y[indices]
        
        for i in range(len(X)):
            n1, a1, n2, a2 = forward(X_shuff[i], W1, W2, b1, b2)
            W1, W2, b1, b2 = backward(X_shuff[i], Y_shuff[i], n1, a1, n2, a2, W1, W2, b1, b2, alpha)
        if(j%100 == 0) :
            print("epochs:", j + 1, "======== Cost:", cost(X, Y, W1, W2, b1, b2))  
    return

### Putting it all together.
Now we have defined all the functions we need. Next, we will use them to build a neural network that takes a single feature, has one hidden layer containing 50 neurons, and then outputs a scalar. 

We will test this model on a toy dataset of $y=\sin(2\pi x)$. First, `np.linspace` is used to make $500$ equally spaced values between 0 and 1, which we will use as our features (500 samples in the dataset, with a single feature per each sample). The line `Y = np.sin(X*2.*np.pi)` then computes the corresponding labels. 

Remember that since we wanted a general code that works with `matmul`, even though we only have one feature and a scalar label, we need each $x$ and each $y$ to be stored as a $(1\times1)$ matrix. The lines `X=X.reshape(-1,1,1)` and `Y=Y.reshape(-1,1,1)` reshape our data accordingly. The $-1$ means we do not explicitly set the first dimension (which will equal $N$ where $N=500$ in this case) but let numpy work it out based on how many numbers the array holds, and the constraints that each one should be a $(1\times1)$ matrix. We could equally have put `X=X.reshape(500,1,1)`, but then we would need to update the code every time our training set size changes. Run the cell below to see if your implementation worked!

In [9]:
N_Neurons = [1, 50, 1] #number of neurons in each layer (input, hidden and output)

W_layer1, W_layer2, b_layer1, b_layer2 = initialise_params( N_Neurons )

np.random.seed(123456)

#prepare some toy data (y=sin(2*pi * x))
X=np.linspace(0,1,500)
Y = np.sin(X*2.*np.pi)
X=X.reshape(-1,1,1)
Y=Y.reshape(-1,1,1)

train(X,Y, W_layer1, W_layer2, b_layer1, b_layer2, alpha=0.001, epoch=6000)

TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable tanh method

### Plotting

Run the cell below to plot the target and prediction. How did you do?

In [ ]:
plt.plot(X.reshape(-1,1),Y.reshape(-1,1), label='target')

pred = np.zeros_like(Y)
for i in range(len(X)):
    _, _, _, pred[i] = forward(X[i], W_layer1, W_layer2, b_layer1, b_layer2)

plt.plot(X.reshape(-1,1),pred.reshape(-1,1), label='prediction')
plt.legend()

### Things to try:

1. Play with the hyperparameters of your network. Does the prediction get worse? I pre-populated the cells with hyperparameters that work quite well after a brief hyperparameter search, but maybe you can get better. Have a play.

2. Try to get your network to predict a different function.

3. Can you implement a deeper network with 2 hidden layers?

4. [Advanced - only recommended if you want a challenge!] Can you implement a deeper network with an arbitrary number of hidden layers? The code should work with `N_Neurons = [a, b, c, d, e]` or `N_Neurons = [a, b, c, d, e, f, g]` without any modification of the code (other than the line that specifies the network architecture, of course!). Your code should work for `a`, `b` etc. as any integer value.

5. Edit the implementation from a regression task to a classification task. Hint: You may use the sigmoid function for the output layer and interpret the output as a probability of the class being that associated with label 1. (This is your coursework assignment).


A common "trick" in the ML community is to implement sigmoid as in the cell below. This helps prevent numerical overflow over the naive implementation of $\sigma(z) = 1 / (1 + e^{(-z)})$, but gives the same result. 

In [ ]:
def sigmoid_with_trick(z):
    if( z>=0. ):
        return 1. / (1. + np.exp(-z))
    else:
        return np.exp(z) / (1. + np.exp(z))

In [ ]:
def naive_sigmoid(z):
    return 1. / (1. + np.exp(-z))

For example, try running the two cells below:

In [ ]:
naive_sigmoid(-1000)

In [ ]:
sigmoid_with_trick(-1000)